# Predictive Restaurant Recommender

In [1]:
import pandas as pd
import numpy as np 

train_customers = pd.read_csv("Train_soulpage/train_customers.csv")
train_locations = pd.read_csv("Train_soulpage/train_locations.csv")
train_orders = pd.read_csv("Train_soulpage/orders.csv")
vendors = pd.read_csv("Train_soulpage/vendors.csv")

C:\Users\siree\AppData\Local\Temp\ipykernel_2516\3270821030.py:6: DtypeWarning: Columns (15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  train_orders = pd.read_csv("Train_soulpage/orders.csv")


In [2]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

### Step 1 — Understanding train_customers Table & Performing Data Cleaning

In [3]:
train_customers.head(2)

,customer_id,gender,dob,status,verified,language,created_at,updated_at
0,TCHWPBT,Male,NaN,1,1,EN,2/7/2023 19:16,2/7/2023 19:16
1,ZGFSYCZ,Male,NaN,1,1,EN,2/9/2023 12:04,2/9/2023 12:04


In [4]:
train_customers.shape

(34674, 8)

In [5]:
train_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34674 entries, 0 to 34673
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   customer_id  34674 non-null  object 
 1   gender       22520 non-null  object 
 2   dob          3046 non-null   float64
 3   status       34674 non-null  int64  
 4   verified     34674 non-null  int64  
 5   language     21099 non-null  object 
 6   created_at   34674 non-null  object 
 7   updated_at   34674 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 2.1+ MB


- customer_id is must — it’s the unique ID used to link orders, locations, and restaurants.

In [6]:
train_customers["customer_id"].nunique()     

34523

In [7]:
# Duplicate customer_id rows
#duplicate_rows = train_customers[train_customers['customer_id'].duplicated(keep=False)]
#duplicate_rows

In [8]:
train_customers["gender"].unique()          # NaN       12157

array(['Male', 'Female', 'Female  ', 'Female   ', 'Female    ', 'Male  ',
       '  ', 'Female ', nan, '?????', 'male'], dtype=object)

In [9]:
train_customers["gender"].value_counts(dropna=False)

gender
Male          17815
NaN           12154
male           2914
Female         1761
Female           13
Male              9
Female            2
Female            2
?????             2
Female            1
                  1
Name: count, dtype: int64

In [10]:
# Clean the column
train_customers["gender"] = (train_customers["gender"]
                .str.strip()                  # Remove extra spaces
                .str.capitalize()             # Capitalize first letter
                .replace(['', '?????'], np.nan)  # Replace empty/unknown with NaN
               )

% Missing = (12157 / 34674) × 100 ≈ 35%

The gender column has 35% missing values
-  existing values are also highly imbalanced (≈ 20738 Male vs 1779 Female).
Because I don’t have access to the actual genders, imputing the missing ones with an “unknown” category could create noise or bias in the model.

- Since gender is not essential for predicting restaurant preferences and provides very weak signal due to its imbalance and missing data, it is safer and cleaner to drop this column.

In [11]:
train_customers["dob"].unique()

array([      nan, 1.957e+03, 1.970e+03, 1.985e+03, 1.900e+03, 2.002e+03,
       1.989e+03, 2.018e+03, 1.998e+03, 1.991e+03, 1.993e+03, 1.990e+03,
       2.003e+03, 2.000e+03, 1.986e+03, 2.019e+03, 1.977e+03, 1.979e+03,
       1.996e+03, 1.995e+03, 2.001e+03, 1.997e+03, 1.999e+03, 1.994e+03,
       1.980e+03, 2.020e+03, 1.987e+03, 1.988e+03, 1.984e+03, 1.981e+03,
       1.983e+03, 1.992e+03, 1.976e+03, 1.973e+03, 2.005e+03, 1.975e+03,
       1.964e+03, 1.967e+03, 1.982e+03, 1.978e+03, 2.006e+03, 2.017e+03,
       2.004e+03, 2.007e+03, 1.000e+00, 1.974e+03, 1.972e+03, 2.013e+03,
       1.966e+03, 2.015e+03, 1.971e+03, 2.009e+03, 2.531e+03, 2.008e+03,
       2.016e+03, 2.014e+03, 1.953e+03, 2.562e+03, 1.956e+03, 1.934e+03,
       1.398e+03, 2.012e+03, 1.888e+03, 1.968e+03, 1.955e+03, 6.760e+02])

In [12]:
train_customers['dob'] = train_customers['dob'].fillna(0)

In [13]:
# dob column ni int format lo ki marchadam
train_customers['dob'] = train_customers['dob'].astype(float).astype(int)

In [14]:
# print(train_customers['dob'].value_counts().to_string())

1. dob has only 3046 non-null values out of 34674 rows → that means over 91% missing.

2. If you use this column:
   
   - You must either drop 91% of customers (bad — you’ll lose almost all data),
   
   - or fill with guesses (which will add noise, not help the model).
   
 So best decision: Drop dob column completely.
It won’t help the recommendation model and will just create noise.

In [15]:
train_customers["status"].unique()

array([1, 0], dtype=int64)

In [16]:
train_customers["status"].value_counts(dropna=False)

status
1    34639
0       35
Name: count, dtype: int64

In [17]:
#train_customers[train_customers["status"]==0]

- status usually shows if the account is active/inactive or blocked.

- It doesn’t tell anything about food preferences or restaurant choices.

- If almost all users have the same status (like all 1), it gives no useful variation.

So: status is not helpful for recommendations → can be dropped. 

In [18]:
train_customers["verified"].unique()

array([1, 0], dtype=int64)

In [19]:
train_customers["verified"].value_counts()

verified
1    33167
0     1507
Name: count, dtype: int64

- verified only shows if the user’s account is verified (like email/phone confirmed).

- This doesn’t affect what restaurants they like or order from.

So: verified is not useful for recommendations → safe to drop.`m

In [20]:
train_customers["language"].unique()

array(['EN', nan], dtype=object)

In [21]:
train_customers["language"].value_counts(dropna=False)   # (almost 40% missing)

language
EN     21099
NaN    13575
Name: count, dtype: int64

1. language mostly has "EN" and some nulls , then there’s no real variation.

2. Columns with almost the same value for everyone don’t help models — they can’t differentiate users.

So: Drop language column.

In [22]:
train_customers["created_at"].nunique()

30242

The created_at column indicates when a customer created their account.

**Thought:**

 - Older customers might have developed preferences for specific restaurants over time.

**Practical Consideration:**

- Restaurant recommendations depend more on location, cuisine, and past order history than on account creation date.

- Just knowing when an account was created provides a very weak signal about current restaurant preferences.

- Preferences can change over time, so an old account does not mean the customer still likes the same restaurants.

**Conclusion:**

Since created_at gives limited value without order behavior, it will be dropped from the dataset.

In [23]:
train_customers["updated_at"].nunique()

22319

- updated_at shows the last profile update, which is not related to restaurant preferences or orders.
It adds no predictive value, so this column will be dropped.

#### Dropping unimportant columns

In [24]:
# Dropping unnecessary columns
train_customers.drop(['gender','dob','status','verified','language','created_at','updated_at'], axis=1, inplace=True)

In [25]:
# Check duplicates
train_customers['customer_id'].duplicated().sum()

151

In [26]:
train_customers.drop_duplicates(subset='customer_id', inplace=True)


In [27]:
train_customers.head(2)

,customer_id
0,TCHWPBT
1,ZGFSYCZ


In [28]:
train_customers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34523 entries, 0 to 34673
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  34523 non-null  object
dtypes: object(1)
memory usage: 539.4+ KB


### Step 2 — Understanding train_locations Table & Performing Data Cleaning

In [29]:
train_locations.head(2)

,customer_id,location_number,location_type,latitude,longitude
0,02SFNJH,0,NaN,1.682392,-78.789737
1,02SFNJH,1,NaN,1.679137,0.766823


In [30]:
train_locations.shape

(59503, 5)

In [31]:
train_locations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59503 entries, 0 to 59502
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      59503 non-null  object 
 1   location_number  59503 non-null  int64  
 2   location_type    32294 non-null  object 
 3   latitude         59497 non-null  float64
 4   longitude        59497 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 2.3+ MB


- Links each location to the customer, necessary for building recommendations. 

In [32]:
train_locations["customer_id"].nunique()       

35400

- One customer_id can appear multiple times with different location_number and location_type.
- This is normal and expected for this dataset.We should not remove duplicates just based on customer_id

- customer_id can repeat
- customer_id + location_number pair must be unique
- That pair will be used later when linking orders and building recommendations

- train_locations table is just a list of all locations a customer has ever used.
- So ideally, each customer + location_number pair should appear only once in this table.

In [33]:
# Let’s now check if any duplicate customer_id + location_number pairs exist in your train_locations table.
dup_count = train_locations.duplicated(subset=['customer_id','location_number']).sum()
print("Number of duplicate pairs:", dup_count)

Number of duplicate pairs: 0


- dup_count shows 0 → No duplicates, nothing to remove.

In [34]:
train_locations["location_number"].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], dtype=int64)

In [35]:
#train_locations[train_locations["location_number"]==4]

- What it is: An index for each customer’s saved locations (0, 1, 2 …).
- Why useful:

   - Customers can have multiple addresses (home, work, other)
   - location_number helps differentiate these locations.
   - Important if your model wants to predict which restaurant the customer will order from at a specific location.

- Why not useless:

   - Dropping it would mix all locations of a customer together → model cannot distinguish which location is relevant.
   - Conclusion: Keep location_number 

In [36]:
# train_locations["location_number"].value_counts()

In [37]:
train_locations["location_type"].unique()           # Not useful ( all missing )

array([nan, 'Home', 'Work', 'Other'], dtype=object)

In [38]:
#train_locations[train_locations["location_type"]=='Home']

In [39]:
train_locations["location_type"].value_counts(dropna=False)

location_type
NaN      27209
Home     19703
Work      6441
Other     6150
Name: count, dtype: int64

- Total rows = 59,503
- Missing (NaN) = 27,209 → ~46% missing

**Observation:**

- Almost half the values are missing → high missing rate.
- Also, the distribution is very uneven: Home dominates, Work & Other are small.
- Models may get biased or noisy signal from this column.

**Decision:**

- Drop location_type 
- Reason: Too many missing values and uneven distribution → weak predictive power for restaurant choice.

In [40]:
train_locations["latitude"].nunique()      # 6 missing 

58454

In [41]:
train_locations["longitude"].nunique()     # 6 missing 

58657

In [42]:
train_locations[train_locations["latitude"].isnull()]

,customer_id,location_number,location_type,latitude,longitude
24726,7URX8JP,0,Other,NaN,NaN
35657,NSQRO1H,2,NaN,NaN,NaN
42953,55MCNEF,0,Home,NaN,NaN
45266,VZIK43C,2,NaN,NaN,NaN
50037,QFWLNUK,2,NaN,NaN,NaN
59198,O0LALCF,0,NaN,NaN,NaN


- Since it’s only 6 out of 59,503 rows, dropping them is the cleanest and safest approach.

In [43]:
# Drop rows where either latitude OR longitude is null
train_locations.dropna(subset=['latitude', 'longitude'], inplace=True)

In [44]:
# Reset index after dropping
train_locations.reset_index(drop=True, inplace=True)

In [45]:
# Confirm there are no nulls now
train_locations[['latitude', 'longitude']].isnull().sum()

latitude     0
longitude    0
dtype: int64

- Exact location coordinates are critical to predict nearby restaurants.
- so am keeping both latitude and logitude  

#### Dropping unimportant columns

In [46]:
train_locations.drop(['location_type'], axis=1, inplace=True)

In [47]:
train_locations.head(2)

,customer_id,location_number,latitude,longitude
0,02SFNJH,0,1.682392,-78.789737
1,02SFNJH,1,1.679137,0.766823


In [48]:
train_locations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59497 entries, 0 to 59496
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      59497 non-null  object 
 1   location_number  59497 non-null  int64  
 2   latitude         59497 non-null  float64
 3   longitude        59497 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 1.8+ MB


### Step 3 — Understanding train_orders Table & Performing Data Cleaning

In [49]:
train_orders.head(2)

,order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,vendor_rating,driver_rating,deliverydistance,preparationtime,delivery_time,order_accepted_time,driver_accepted_time,ready_for_pickup_time,picked_up_time,delivered_time,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR
0,163923.0,KL09J9N,6.0,10.1,1,NaN,0.0,NaN,NaN,No,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/1/2024 5:30,84,8/2/2024 5:33,0,Work,KL09J9N X 0 X 84
1,163924.0,H5LGGFX,3.0,8.4,1,NaN,0.0,NaN,NaN,No,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/1/2024 5:30,78,8/2/2024 5:34,0,Home,H5LGGFX X 0 X 78


In [50]:
train_orders.shape


(135303, 26)

In [51]:
train_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135303 entries, 0 to 135302
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   order_id                        135233 non-null  float64
 1   customer_id                     135303 non-null  object 
 2   item_count                      128378 non-null  float64
 3   grand_total                     135303 non-null  float64
 4   payment_mode                    135303 non-null  int64  
 5   promo_code                      4305 non-null    object 
 6   vendor_discount_amount          135303 non-null  float64
 7   promo_code_discount_percentage  65880 non-null   float64
 8   is_favorite                     100108 non-null  object 
 9   is_rated                        135303 non-null  object 
 10  vendor_rating                   45220 non-null   float64
 11  driver_rating                   135303 non-null  int64  
 12  deliverydistance

In [52]:
train_orders['order_id'].nunique()

135221

In [53]:
#This will show which order_ids appear more than once and how many times each appears.
train_orders['order_id'].value_counts()[train_orders['order_id'].value_counts() > 1]

order_id
230681.0    2
230638.0    2
251012.0    2
230637.0    2
230686.0    2
253161.0    2
230684.0    2
230682.0    2
251897.0    2
251526.0    2
253208.0    2
252730.0    2
Name: count, dtype: int64

In [54]:
duplicate_order_ids = train_orders['order_id'][train_orders['order_id'].duplicated()]
duplicate_order_ids

48307     230637.0
48308     230638.0
48329     230681.0
48330     230682.0
48331     230684.0
            ...   
112974         NaN
117886         NaN
117887         NaN
117890         NaN
118113         NaN
Name: order_id, Length: 81, dtype: float64

In [55]:
# Get all rows for those duplicated order_ids
duplicated_orders_df = train_orders[train_orders['order_id'].isin(duplicate_order_ids)]
duplicated_orders_df.head(1)

,order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,vendor_rating,driver_rating,deliverydistance,preparationtime,delivery_time,order_accepted_time,driver_accepted_time,ready_for_pickup_time,picked_up_time,delivered_time,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR
48295,230681.0,Z6DDRUJ,1.0,13.3,2,NaN,0.0,NaN,No,No,0.0,0,0.0,NaN,NaN,10/21/2019 11:50,NaN,10/21/2019 12:02,10/21/2019 16:03,10/21/2019 16:16,NaN,134,10/21/2024 15:49,2,Work,Z6DDRUJ X 2 X 134


- order_id is supposed to be unique for every order.
- In the dataset, you saw that some order_ids appear twice.
- These duplicated order_ids are not identical rows:
  - One row may have some missing values (partially filled)
  - Another row may have more complete information
  - Timestamps or other columns may differ slightly

In [56]:
# Sort by order_id to group duplicates together
duplicated_orders_df = duplicated_orders_df.sort_values(by='order_id')
duplicated_orders_df.head(1)

,order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,vendor_rating,driver_rating,deliverydistance,preparationtime,delivery_time,order_accepted_time,driver_accepted_time,ready_for_pickup_time,picked_up_time,delivered_time,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR
48299,230637.0,2LWS99M,NaN,7.3,2,NaN,0.0,NaN,No,No,0.0,0,0.0,NaN,NaN,10/21/2019 12:00,NaN,10/21/2019 12:00,10/21/2019 16:01,10/21/2019 16:01,NaN,4,10/21/2024 11:57,1,NaN,2LWS99M X 1 X 4


- order_id is supposed to be unique for every order placed.
- One customer can place many orders, so same customer_id repeats — this is fine.
- But same order_id repeating means:
   - The same order is appearing more than once in the dataset.
   - These are true duplicate rows (or at least duplicate order entries).

- So yes —
   - If order_id repeats → those are duplicates → keep only one and drop the rest 

In [57]:
# Remove NaNs first (if any)
train_orders = train_orders.dropna(subset=['order_id'])

In [58]:
# Keep only the first row for each order_id
train_orders = train_orders.drop_duplicates(subset=['order_id'], keep='first')

In [59]:
print(train_orders['order_id'].nunique(), len(train_orders))

135221 135221


In [60]:
has_duplicates = train_orders['order_id'].duplicated().any()
print("Are there duplicated order_ids?", has_duplicates)


Are there duplicated order_ids? False


In [61]:
 #Step 2: Count how many duplicated order_ids
num_duplicates = train_orders['order_id'].duplicated().sum()
print("Number of duplicated order_id rows:", num_duplicates)


Number of duplicated order_id rows: 0


- The dataset contains several time-related columns, including preparationtime, delivery_time, order_accepted_time, driver_accepted_time, ready_for_pickup_time, picked_up_time, and delivered_time. While these columns theoretically influence a customer’s choice—since long delivery times may discourage repeat orders—the majority of their values are missing.
-  For example, delivery_time has only 5,123 non-null entries out of 135,303 orders. This makes it extremely difficult to reliably compute a feature like total delivery time. Given the sparsity and the complexity of imputing or deriving meaningful metrics from these columns, it is more practical to drop them.
-  Instead, we focus on more informative features such as customer preferences, vendor ratings, and delivery distance, which are more consistently populated and directly relevant for predicting the best restaurant for each customer.

In [62]:
#train_orders[train_orders['promo_code_discount_percentage']==25][['promo_code']]

In [63]:
train_orders.drop(['order_id',"payment_mode","promo_code","driver_rating","promo_code_discount_percentage","created_at","delivery_date",'LOCATION_TYPE','CID X LOC_NUM X VENDOR','preparationtime',
       'delivery_time', 'order_accepted_time', 'driver_accepted_time',
       'ready_for_pickup_time', 'picked_up_time', 'delivered_time'],axis=1,inplace=True)

- here in this dataset Most customers don’t use promo codes when ordering.
- That’s why promo_code has only 4305 non-null values out of 135303 orders (~3%).
- For your restaurant recommendation project, this column is not useful because:
   - It doesn’t tell you anything about restaurant preference
  - Too sparse → filling it or keeping it may add noise
- So, dropping promo_code is the right decision.

In [64]:
train_orders.head(1)

,customer_id,item_count,grand_total,vendor_discount_amount,is_favorite,is_rated,vendor_rating,deliverydistance,vendor_id,LOCATION_NUMBER
0,KL09J9N,6.0,10.1,0.0,NaN,No,NaN,0.0,84,0


In [65]:
train_orders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135221 entries, 0 to 135302
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   customer_id             135221 non-null  object 
 1   item_count              128302 non-null  float64
 2   grand_total             135221 non-null  float64
 3   vendor_discount_amount  135221 non-null  float64
 4   is_favorite             100032 non-null  object 
 5   is_rated                135221 non-null  object 
 6   vendor_rating           45220 non-null   float64
 7   deliverydistance        135221 non-null  float64
 8   vendor_id               135221 non-null  int64  
 9   LOCATION_NUMBER         135221 non-null  int64  
dtypes: float64(5), int64(2), object(3)
memory usage: 11.3+ MB


In [66]:
train_orders['item_count'].unique()        # 6925 null values 

array([ 6.,  3.,  4.,  7.,  1.,  2.,  5.,  8., 10.,  9., 11., 15., 14.,
       16., 12., 19., 13., 17., 18., 27., 20., 34., 22., 21., 47., 41.,
       26., 24., 28., 25., nan, 40., 33., 30., 68., 38.])

In [67]:
#train_orders['item_count'].value_counts(dropna=False)    # NaN      6919   -->5% missing 

In [68]:
train_orders[train_orders['item_count']==68][['item_count','grand_total']]

,item_count,grand_total
94257,68.0,783.8




**item_count**

- What it is: Number of items in a single order.
- Why it matters for your recommendation engine:
  - Shows how much a customer usually orders
  - Can indicate favorite order size → some customers order 1–2 items, others 5–10 items
  - Helps model customer behavior for recommending vendors who usually serve that order size

- Missing value → fill with median (robust to outliers)
- Outliers (like 68 items) → we can keep them for now unless they are clearly wrong

In [69]:
# Fill missing with median
median_item_count = train_orders['item_count'].median()
train_orders['item_count'] = train_orders['item_count'].fillna(median_item_count)

In [70]:
#print(train_orders['grand_total'].unique().tolist())
#print(sorted(train_orders['grand_total'].unique()))

In [71]:
# len(train_orders[train_orders['grand_total']==0])   # 647
train_orders.loc[train_orders['grand_total'] == 0, 'item_count'].unique()

array([1., 2., 3., 4., 7., 5., 6., 8., 9.])

In [72]:
train_orders['vendor_discount_amount'].unique()

array([ 0.   ,  0.84 ,  1.05 ,  0.75 ,  0.33 ,  0.435,  0.15 ,  2.04 ,
        0.39 ,  0.615,  1.125,  0.375,  0.825,  0.72 ,  1.065,  0.63 ,
        0.255,  0.78 ,  0.765,  1.185,  0.465,  1.035,  0.81 ,  0.3  ,
        0.405,  0.21 ,  1.44 ,  0.525,  0.24 ,  0.555,  0.495,  0.87 ,
        0.975,  0.51 ,  0.96 ,  0.66 ,  0.585,  3.18 ,  0.45 ,  0.195,
        0.42 ,  0.18 ,  0.225,  0.48 ,  0.9  ,  2.95 ,  3.625,  0.69 ,
        3.825,  1.175,  1.08 ,  1.305,  0.315,  3.05 ,  0.93 ,  0.675,
        4.95 ,  0.6  , 10.65 ,  2.1  ,  2.2  ,  0.36 ,  4.1  ,  0.165,
        0.345,  0.54 ,  0.425,  0.12 ,  0.075,  0.99 ,  1.25 ,  0.705,
        2.085,  0.38 ,  1.38 ,  0.57 ,  1.14 ,  1.785])

In [73]:
train_orders['is_favorite'].unique()

array([nan, 'No', 'Yes'], dtype=object)

In [74]:
train_orders['is_favorite'].value_counts(dropna=False)

is_favorite
No     98373
NaN    35189
Yes     1659
Name: count, dtype: int64

- Reason: Most users haven’t marked favorites, so missing probably means “not favorite”.

In [75]:
train_orders['is_favorite'] = train_orders['is_favorite'].fillna('No')

In [76]:
train_orders['is_rated'].unique()

array(['No', 'Yes'], dtype=object)

In [77]:
train_orders['is_rated'].value_counts(dropna=False)

is_rated
No     115112
Yes     20109
Name: count, dtype: int64

In [78]:
# Convert Yes/No to 1/0
train_orders['is_favorite'] = train_orders['is_favorite'].map({'Yes': 1, 'No': 0})
train_orders['is_rated'] = train_orders['is_rated'].map({'Yes': 1, 'No': 0})

In [79]:
train_orders['vendor_rating'].unique()       # 90083 null values

array([nan,  5.,  2.,  1.,  4.,  3.,  0.])

In [80]:
train_orders['vendor_rating'].value_counts(dropna=False)  

vendor_rating
NaN    90001
0.0    25175
5.0    14212
4.0     2748
3.0     1426
1.0     1029
2.0      630
Name: count, dtype: int64

- The vendor_rating column has 66% missing values, which is very high.
- Missing rating means the user did not rate the vendor.
- In this case, it is reasonable to assume that unrated vendors get a rating of 0.
- To preserve information, we can also create a flag column has_vendor_rating:
   - 1 → vendor has a rating
  - 0 → vendor has no rating
- This way, the model can distinguish between vendors that are rated and those that are not, while still having a usable vendor_rating column.

In [81]:
# Mark which vendors have ratings
train_orders['has_vendor_rating'] = train_orders['vendor_rating'].notna().astype(int)

In [82]:
# Fill missing vendor ratings with 0
train_orders['vendor_rating'].fillna(0, inplace=True)

In [83]:
a=train_orders[['is_rated','vendor_rating']]
#a[a['vendor_rating'].isnull()].shape[0]   # 90083
a[a['vendor_rating'].isnull()]['is_rated'].unique()

array([], dtype=int64)

In [84]:
train_orders['deliverydistance'].unique()

array([0.  , 1.68, 3.87, ..., 0.23, 0.05, 0.06])

In [85]:
#pd.set_option('display.max_rows', None)

In [86]:
'''
# Convert to datetime
train_orders['order_accepted_time'] = pd.to_datetime(train_orders['order_accepted_time'], errors='coerce')
train_orders['delivered_time'] = pd.to_datetime(train_orders['delivered_time'], errors='coerce')

# Calculate duration in minutes
train_orders['delivery_duration_min'] = (train_orders['delivered_time'] - train_orders['order_accepted_time']).dt.total_seconds() / 60

# Function to convert minutes to H:MM
def minutes_to_h_mm(minutes):
    if pd.isna(minutes):
        return np.nan
    hours = int(minutes // 60)
    mins = int(minutes % 60)
    return f"{hours}:{mins:02d}"

# Create single column in H:MM format
train_orders['delivery_duration_h_mm'] = train_orders['delivery_duration_min'].apply(minutes_to_h_mm)

# Optional: drop the intermediate minutes column
train_orders.drop(columns=['delivery_duration_min'], inplace=True)
'''

'\n# Convert to datetime\ntrain_orders[\'order_accepted_time\'] = pd.to_datetime(train_orders[\'order_accepted_time\'], errors=\'coerce\')\ntrain_orders[\'delivered_time\'] = pd.to_datetime(train_orders[\'delivered_time\'], errors=\'coerce\')\n\n# Calculate duration in minutes\ntrain_orders[\'delivery_duration_min\'] = (train_orders[\'delivered_time\'] - train_orders[\'order_accepted_time\']).dt.total_seconds() / 60\n\n# Function to convert minutes to H:MM\ndef minutes_to_h_mm(minutes):\n    if pd.isna(minutes):\n        return np.nan\n    hours = int(minutes // 60)\n    mins = int(minutes % 60)\n    return f"{hours}:{mins:02d}"\n\n# Create single column in H:MM format\ntrain_orders[\'delivery_duration_h_mm\'] = train_orders[\'delivery_duration_min\'].apply(minutes_to_h_mm)\n\n# Optional: drop the intermediate minutes column\ntrain_orders.drop(columns=[\'delivery_duration_min\'], inplace=True)\n'

deliverydistance column:

- What it is: The distance (usually in km or miles) between the customer’s location and the restaurant for that order.
- Why it’s useful:

  - Customers usually prefer restaurants closer to them.
  - Can be used to predict the likelihood of a customer choosing a restaurant.
  - Can help the model learn patterns like “customer rarely orders from restaurants beyond X km.”
- Why not useless:

  - Unlike timestamps or promo codes, distance directly affects restaurant choice.
  - Important for location-based recommendations.

Conclusion: Keep deliverydistance 

preparationtime

What it is: Time (in minutes) the restaurant takes to prepare an order.

Why it’s useful: Customers may prefer restaurants that prepare food faster; can influence their choice.

Problem: ~41% missing values — filling them blindly could add noise or bias.

Safe Option: Calculate average preparation time per restaurant and use that as a feature.

Conclusion: Keep aggregated average per restaurant; drop the raw preparationtime column for baseline. 

In [87]:
train_orders['LOCATION_NUMBER'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  8,  7,  9, 11, 10, 13, 15, 16, 19, 21,
       23, 24, 25, 22, 12, 14, 17], dtype=int64)

In [88]:
#train_orders['LOCATION_TYPE'].unique()    # 48893 null values 

In [89]:
#train_orders['CID X LOC_NUM X VENDOR'].unique()    

In [90]:
train_orders.head(2)

,customer_id,item_count,grand_total,vendor_discount_amount,is_favorite,is_rated,vendor_rating,deliverydistance,vendor_id,LOCATION_NUMBER,has_vendor_rating
0,KL09J9N,6.0,10.1,0.0,0,0,0.0,0.0,84,0,0
1,H5LGGFX,3.0,8.4,0.0,0,0,0.0,0.0,78,0,0


In [91]:
train_orders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135221 entries, 0 to 135302
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   customer_id             135221 non-null  object 
 1   item_count              135221 non-null  float64
 2   grand_total             135221 non-null  float64
 3   vendor_discount_amount  135221 non-null  float64
 4   is_favorite             135221 non-null  int64  
 5   is_rated                135221 non-null  int64  
 6   vendor_rating           135221 non-null  float64
 7   deliverydistance        135221 non-null  float64
 8   vendor_id               135221 non-null  int64  
 9   LOCATION_NUMBER         135221 non-null  int64  
 10  has_vendor_rating       135221 non-null  int32  
dtypes: float64(5), int32(1), int64(4), object(1)
memory usage: 11.9+ MB


### Step 4 — Understanding vendors Table & Performing Data Cleaning

In [92]:
vendors.head(2)

,id,authentication_id,latitude,longitude,vendor_category_en,vendor_category_id,delivery_charge,serving_distance,is_open,OpeningTime,OpeningTime2,prepration_time,commission,is_haked_delivering,discount_percentage,status,verified,rank,language,vendor_rating,sunday_from_time1,sunday_to_time1,sunday_from_time2,sunday_to_time2,monday_from_time1,monday_to_time1,monday_from_time2,monday_to_time2,tuesday_from_time1,tuesday_to_time1,tuesday_from_time2,tuesday_to_time2,wednesday_from_time1,wednesday_to_time1,wednesday_from_time2,wednesday_to_time2,thursday_from_time1,thursday_to_time1,thursday_from_time2,thursday_to_time2,friday_from_time1,friday_to_time1,friday_from_time2,friday_to_time2,saturday_from_time1,saturday_to_time1,saturday_from_time2,saturday_to_time2,primary_tags,open_close_flags,vendor_tag,vendor_tag_name,one_click_vendor,country_id,city_id,created_at,updated_at,device_type,display_orders
0,4,118597,-0.588596,0.754434,Restaurants,2,0.0,6,1,11:00AM-11:30PM,-,15,0.0,Yes,0,1,1,11,EN,4.4,0:00:00,0:30:00,8:00:00,23:59:00,0:00:00,0:30:00,8:00:00,23:59:00,0:00:00,0:30:00,8:00:00,23:59:00,0:00:00,0:30:00,8:00:00,23:59:00,0:00:00,0:30:00,8:00:00,23:59:00,0:00:00,0:30:00,10:00:00,23:59:00,0:00:00,0:30:00,10:00:00,23:59:00,"{""primary_tags"":""4""}",1,"2,4,5,8,91,22,12,24,16,23","Arabic,Breakfast,Burgers,Desserts,Free Deliver...",Y,1,1,1/30/2023 14:42,4/7/2025 15:12,3,1
1,13,118608,-0.471654,0.744470,Restaurants,2,0.7,5,1,08:30AM-10:30PM,-,14,0.0,Yes,0,1,1,11,EN,4.7,0:00:00,1:30:00,8:00:00,23:59:00,0:00:00,1:30:00,8:00:00,23:59:00,0:00:00,1:30:00,8:00:00,23:59:00,0:00:00,1:30:00,8:00:00,19:30:00,0:00:00,1:30:00,8:00:00,19:30:00,0:00:00,1:30:00,8:00:00,23:59:00,0:00:00,1:30:00,8:00:00,23:59:00,"{""primary_tags"":""7""}",1,"4,41,51,34,27,15,24,16,28","Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa...",Y,1,1,5/3/2023 12:32,4/5/2025 20:46,3,1


In [93]:
vendors.shape

(100, 59)

In [94]:
vendors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 59 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    100 non-null    int64  
 1   authentication_id     100 non-null    int64  
 2   latitude              100 non-null    float64
 3   longitude             100 non-null    float64
 4   vendor_category_en    100 non-null    object 
 5   vendor_category_id    100 non-null    int64  
 6   delivery_charge       100 non-null    float64
 7   serving_distance      100 non-null    int64  
 8   is_open               100 non-null    int64  
 9   OpeningTime           91 non-null     object 
 10  OpeningTime2          91 non-null     object 
 11  prepration_time       100 non-null    int64  
 12  commission            85 non-null     float64
 13  is_haked_delivering   100 non-null    object 
 14  discount_percentage   100 non-null    int64  
 15  status                10

In [95]:
vendors.drop([
    'sunday_from_time1','sunday_to_time1','sunday_from_time2','sunday_to_time2',
    'monday_from_time1','monday_to_time1','monday_from_time2','monday_to_time2',
    'tuesday_from_time1','tuesday_to_time1','tuesday_from_time2','tuesday_to_time2',
    'wednesday_from_time1','wednesday_to_time1','wednesday_from_time2','wednesday_to_time2',
    'thursday_from_time1','thursday_to_time1','thursday_from_time2','thursday_to_time2',
    'friday_from_time1','friday_to_time1','friday_from_time2','friday_to_time2',
    'saturday_from_time1','saturday_to_time1','saturday_from_time2','saturday_to_time2',
    "authentication_id",'commission','is_haked_delivering','open_close_flags',
    'one_click_vendor','device_type','status','vendor_tag','updated_at'
], axis=1, inplace=True, errors='ignore')


In this project, our goal is to build a predictive restaurant recommender that identifies which restaurant a customer is most likely to order from based on their preferences, location, and order history. Therefore, we focus only on columns that provide meaningful information for predicting customer choice. Columns such as commission, is_haked_delivering, open_close_flags, one_click_vendor, device_type, status, vendor_tag, and updated_at are removed because they represent internal operational details, redundant information, or system-level flags that have no direct impact on a customer’s decision. Removing these columns helps simplify the dataset, reduce noise, and ensures that the model only learns from relevant features such as restaurant location, category, availability, ratings, and tags.

1.commission – Percentage fee the restaurant pays to the app. This is internal business info and does not affect customer preference.

2.is_haked_delivering – Indicates if delivery is handled via third-party or special method. Completely internal operational detail, irrelevant to the recommendation.

3.open_close_flags – Internal flag showing open/closed status. Already captured by is_open; keeping both is redundant.

4.one_click_vendor – Whether the restaurant supports one-click ordering. A UX feature; does not influence customer choice.

5.device_type – The type of device the vendor uses. Purely internal tech info; doesn’t affect orders.

6.status – Vendor account status (active/inactive). Already captured by is_open and the dataset filters; not predictive.

7.vendor_tag – Extra vendor tags, but vendor_tag_name already contains meaningful descriptive tags. Redundant.

8.updated_at – Timestamp of last vendor info update. Used only for record-keeping; has no predictive value.

In [96]:
vendors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   100 non-null    int64  
 1   latitude             100 non-null    float64
 2   longitude            100 non-null    float64
 3   vendor_category_en   100 non-null    object 
 4   vendor_category_id   100 non-null    int64  
 5   delivery_charge      100 non-null    float64
 6   serving_distance     100 non-null    int64  
 7   is_open              100 non-null    int64  
 8   OpeningTime          91 non-null     object 
 9   OpeningTime2         91 non-null     object 
 10  prepration_time      100 non-null    int64  
 11  discount_percentage  100 non-null    int64  
 12  verified             100 non-null    int64  
 13  rank                 100 non-null    int64  
 14  language             85 non-null     object 
 15  vendor_rating        100 non-null    floa

In [97]:
vendors['id'].nunique()

100

In [98]:
vendors['vendor_category_en'].unique()  

array(['Restaurants', 'Sweets & Bakes'], dtype=object)

In [99]:
vendors['vendor_category_en'].value_counts() 

vendor_category_en
Restaurants       88
Sweets & Bakes    12
Name: count, dtype: int64

In [100]:
vendors["vendor_category_id"].unique()

array([2, 3], dtype=int64)

In [101]:
vendors[vendors['vendor_category_en']=='Restaurants']['vendor_category_id'].unique()

array([2], dtype=int64)

In [102]:
vendors[vendors['vendor_category_en']!='Restaurants'][["vendor_category_en","vendor_category_id"]]

,vendor_category_en,vendor_category_id
19,Sweets & Bakes,3
20,Sweets & Bakes,3
28,Sweets & Bakes,2
46,Sweets & Bakes,3
48,Sweets & Bakes,3
53,Sweets & Bakes,3
55,Sweets & Bakes,3
60,Sweets & Bakes,3
62,Sweets & Bakes,3
74,Sweets & Bakes,3



Observation on Vendor Categories

1. All vendors with vendor_category_en = 'Restaurants' have vendor_category_id = 2.

2. Most vendors with vendor_category_en = 'Sweets & Bakes' have vendor_category_id = 3, with only one entry showing 2, which seems to be a data entry error.

3. This confirms that vendor_category_en is more reliable than vendor_category_id for identifying vendor type, because the IDs may have occasional inconsistencies.

4. For our rcommender, we will use vendor_category_en to filter Restaurants (2) and Sweets & Bakes (3), ensuring accurate categorization.

In [103]:
# Assume vendors table name is vendors
vendors = pd.get_dummies(vendors, columns=['vendor_category_en'], drop_first=True)

In [104]:
vendors.head(1)

,id,latitude,longitude,vendor_category_id,delivery_charge,serving_distance,is_open,OpeningTime,OpeningTime2,prepration_time,discount_percentage,verified,rank,language,vendor_rating,primary_tags,vendor_tag_name,country_id,city_id,created_at,display_orders,vendor_category_en_Sweets & Bakes
0,4,-0.588596,0.754434,2,0.0,6,1,11:00AM-11:30PM,-,15,0,1,11,EN,4.4,"{""primary_tags"":""4""}","Arabic,Breakfast,Burgers,Desserts,Free Deliver...",1,1,1/30/2023 14:42,1,False


In [105]:
vendors["delivery_charge"].unique()      # Users care about cost to get food.

array([0. , 0.7])

In [106]:
vendors["delivery_charge"].value_counts()

delivery_charge
0.7    59
0.0    41
Name: count, dtype: int64

Observation on Delivery Charge

The delivery_charge column has only two unique values: 0.0 and 0.7.

This indicates that some vendors offer free delivery, while others charge a flat fee of 0.7.

Since there are only two possible values, this feature is low variability, but it can still be useful if you want the model to consider free vs paid delivery in predicting customer choices.

In [107]:
vendors["serving_distance"].unique()    # Helps predict if a restaurant is feasible for delivery.

array([ 6,  5,  8, 15, 10,  2, 14,  7,  3, 12], dtype=int64)

In [108]:
vendors["serving_distance"].value_counts()

serving_distance
15    57
10    12
5     10
6      8
8      8
2      1
14     1
7      1
3      1
12     1
Name: count, dtype: int64

In [109]:
vendors["is_open"].unique()

array([1, 0], dtype=int64)

In [110]:
vendors["is_open"].value_counts()

is_open
1    85
0    15
Name: count, dtype: int64

In [111]:
vendors["OpeningTime"].unique()      # Optional but can help check if restaurant is open at order time.

array(['11:00AM-11:30PM', '08:30AM-10:30PM', '08:00AM-10:45PM',
       '10:59AM-10:30PM', '11:00AM-11:45PM', '11:00AM-10:30PM',
       '09:00AM-11:30PM', '05:00PM-11:00PM', '08:00AM-11:30PM',
       '08:30AM-09:30PM', '11:00AM-11:00PM', '11:59AM-2:15 am',
       '08:00AM-12:30PM', '08:00AM-11:59PM', '10:00AM-11:45PM',
       '11:59AM-11:30PM', '08:30AM-11:45PM', '11:59AM-11:45PM',
       '11:00AM-10:45PM', '10:59AM-11:30PM', '11:15AM-10:00PM',
       '10:59AM-10:59PM', '09:59AM-11:45PM', '8:00AM-09:45PM',
       '04:00PM-11:45PM', '08:00AM-11:45PM', '11:15AM-10:30PM',
       '10:59AM-3:30PM', '11.30am-11:30PM', '11:00AM-11:59PM',
       '09:00AM-09:01AM', '11:59AM-10:45PM', '7:58AM-11:45PM',
       '09:00 AM-11:45 PM', '11:00AM-11:01PM', '05:30PM-11:59PM',
       '06:00PM-11:59PM', '11:59AM-10:45PM ', '11:00AM-3:50pm',
       '11:00AM-04:00PM', '09:00AM-08:00pm ', '09:00AM-11:00PM',
       '06:15AM-11:45PM', '9am-10pm', '08:30PM-11:59PM',
       '01.00PM-02:.00AM', '09:00AM-11:45PM', '

In [112]:
vendors["OpeningTime2"].unique()

array(['-', '05:00PM-11:30PM', '06:30PM-11:00PM', '12:00AM-01:00AM',
       '00:01AM-02:30AM', '6:10pm-12:00am', '06:00PM-11:45PM',
       '10pm-11pm', '01.00PM-02.00AM', '05:15PM-09:30PM ',
       '09:00AM-11:59PM', nan], dtype=object)

In [113]:
vendors["prepration_time"].unique()       # Delivery depends on prep time, so useful.

array([15, 14, 19, 16, 10, 17, 20, 11, 13, 12, 18, 21, 45,  5],
      dtype=int64)

In [114]:
vendors["prepration_time"].value_counts()  

prepration_time
15    36
10    25
14     7
17     6
11     5
13     5
16     4
20     3
19     2
12     2
18     2
21     1
45     1
5      1
Name: count, dtype: int64

In [115]:
vendors["discount_percentage"].unique()     # Some users prefer discounted restaurants.

array([ 0, 10, 50, 35, 15], dtype=int64)

In [116]:
vendors["discount_percentage"].value_counts() 

discount_percentage
0     96
10     1
50     1
35     1
15     1
Name: count, dtype: int64

In [117]:
vendors["verified"].unique()

array([1, 0], dtype=int64)

In [118]:
vendors["verified"].value_counts()

verified
1    88
0    12
Name: count, dtype: int64

In [119]:
vendors["rank"].unique()    # Might indicate popularity or quality.

array([11,  1], dtype=int64)

In [120]:
vendors["rank"].value_counts()

rank
11    89
1     11
Name: count, dtype: int64

In [121]:
vendors["language"].unique()    # can drop , all seams EN , and some dropped , they also might be EN 

array(['EN', nan], dtype=object)

In [122]:
vendors["vendor_rating"].unique()    # Shows quality/reputation.

array([4.4, 4.7, 4.5, 4.6, 4.3, 4. , 3.8, 4.2, 4.8, 3.2, 4.1, 3.7])

In [123]:
vendors["vendor_rating"].value_counts() 

vendor_rating
4.5    19
4.3    18
4.2    15
4.4    14
4.6    10
4.7     7
4.0     6
4.1     5
3.8     2
4.8     2
3.2     1
3.7     1
Name: count, dtype: int64

In [124]:
vendors["primary_tags"].unique()

array(['{"primary_tags":"4"}', '{"primary_tags":"7"}',
       '{"primary_tags":"71"}', '{"primary_tags":"46"}',
       '{"primary_tags":"32"}', '{"primary_tags":"51"}',
       '{"primary_tags":"8"}', '{"primary_tags":"21"}',
       '{"primary_tags":"271"}', '{"primary_tags":"129"}', nan,
       '{"primary_tags":"24"}', '{"primary_tags":"323"}',
       '{"primary_tags":"134"}', '{"primary_tags":"90"}',
       '{"primary_tags":"336"}', '{"primary_tags":"31"}',
       '{"primary_tags":"462"}', '{"primary_tags":"138"}',
       '{"primary_tags":"534"}', '{"primary_tags":"297"}',
       '{"primary_tags":"35"}', '{"primary_tags":"695"}',
       '{"primary_tags":"701"}', '{"primary_tags":"270"}',
       '{"primary_tags":"52"}', '{"primary_tags":"815"}',
       '{"primary_tags":"849"}', '{"primary_tags":"346"}',
       '{"primary_tags":"47"}', '{"primary_tags":"28"}',
       '{"primary_tags":"180"}', '{"primary_tags":"204"}',
       '{"primary_tags":"260"}', '{"primary_tags":"1088"}',
       '{

In [125]:
# Extract numbers from primary_tags, keep as float to handle NaN
vendors['primary_tags_number'] = vendors['primary_tags'].str.extract(r'"primary_tags":"(\d+)"')[0].astype(float)

# Check result
vendors[['primary_tags', 'primary_tags_number']].head()


,primary_tags,primary_tags_number
0,"{""primary_tags"":""4""}",4.0
1,"{""primary_tags"":""7""}",7.0
2,"{""primary_tags"":""71""}",71.0
3,"{""primary_tags"":""46""}",46.0
4,"{""primary_tags"":""32""}",32.0


In [126]:
#vendors["vendor_tag_name"].unique()

In [127]:
vendors["country_id"].unique()

array([1], dtype=int64)

In [128]:
vendors["city_id"].unique()

array([1], dtype=int64)

In [129]:
vendors["created_at"].unique()

array(['1/30/2023 14:42', '5/3/2023 12:32', '5/4/2023 22:28',
       '5/6/2023 19:20', '5/17/2023 22:12', '5/24/2023 17:23',
       '6/20/2023 12:28', '6/20/2023 13:11', '7/22/2023 13:18',
       '8/9/2023 19:26', '8/9/2023 23:41', '8/22/2023 15:10',
       '8/26/2023 14:45', '8/26/2023 21:47', '8/26/2023 22:36',
       '9/1/2023 20:31', '9/3/2023 19:30', '9/13/2023 20:40',
       '9/16/2023 19:37', '9/16/2023 20:01', '9/20/2023 19:39',
       '10/4/2023 19:41', '10/7/2023 17:26', '10/18/2023 21:01',
       '10/19/2023 12:38', '10/19/2023 13:35', '10/21/2023 22:21',
       '10/24/2023 18:11', '10/28/2023 13:34', '11/27/2023 15:33',
       '12/12/2023 15:39', '12/30/2023 20:20', '1/6/2024 20:42',
       '1/17/2024 11:43', '1/19/2024 14:01', '1/23/2024 15:21',
       '1/28/2024 20:37', '1/30/2024 18:26', '2/23/2024 18:55',
       '2/25/2024 15:30', '3/3/2024 21:04', '3/4/2024 12:31',
       '3/5/2024 20:06', '3/7/2024 15:06', '3/9/2024 19:22',
       '3/10/2024 13:05', '3/10/2024 13:08',

In [130]:
vendors["display_orders"].unique()

array([1], dtype=int64)

##### some more unimportant columns dropping

1. vendor_category_id → Mostly redundant because vendor_category_en is already descriptive and more reliable. Some IDs may be inconsistent.

2. OpeningTime & OpeningTime2 → already captured by is_open for recommendations

3. country_id / city_id → These columns have the same value for all rows, so they provide no useful information for distinguishing vendors.

4. display_orders → All vendors have the same value (1), meaning it has no variation and cannot help in prediction.

5. language → Almost all vendors use English or have null values. This feature is not informative for predicting customer choices.

6. create_at : Customers don’t choose restaurants based on when they joined. Only location, cuisine, rating, and prep time matter.

In [131]:
vendors.drop(["vendor_category_id",'OpeningTime','OpeningTime2',"country_id","city_id","display_orders","language","created_at",'primary_tags'], axis=1,inplace=True)

In [132]:
vendors.head(2)

,id,latitude,longitude,delivery_charge,serving_distance,is_open,prepration_time,discount_percentage,verified,rank,vendor_rating,vendor_tag_name,vendor_category_en_Sweets & Bakes,primary_tags_number
0,4,-0.588596,0.754434,0.0,6,1,15,0,1,11,4.4,"Arabic,Breakfast,Burgers,Desserts,Free Deliver...",False,4.0
1,13,-0.471654,0.744470,0.7,5,1,14,0,1,11,4.7,"Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa...",False,7.0


1. OpeningTime & OpeningTime2 → Restaurant opening hours don’t influence which restaurant a customer will choose, so not needed for recommendations.
2. primary_tags → These tags are extra descriptive info (like cuisine or offers) but we already have vendor_category_en and vendor_tag_name, which are enough.

Summary: These columns are not relevant for predicting customer orders, so we drop them to focus only on meaningful features.

In [133]:
vendors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 100 non-null    int64  
 1   latitude                           100 non-null    float64
 2   longitude                          100 non-null    float64
 3   delivery_charge                    100 non-null    float64
 4   serving_distance                   100 non-null    int64  
 5   is_open                            100 non-null    int64  
 6   prepration_time                    100 non-null    int64  
 7   discount_percentage                100 non-null    int64  
 8   verified                           100 non-null    int64  
 9   rank                               100 non-null    int64  
 10  vendor_rating                      100 non-null    float64
 11  vendor_tag_name                    97 non-null     object 


- out of all avaialable column from vendors table , we have some nulls in vendor_tag_name(3) and primary_tags_number(23).

In [134]:
# vendors['vendor_tag_name'].isnull().sum()    # 3
vendors['primary_tags_number'].isnull().sum()   # 23

23

In [135]:
vendors[vendors['vendor_tag_name'].isnull()]

,id,latitude,longitude,delivery_charge,serving_distance,is_open,prepration_time,discount_percentage,verified,rank,vendor_rating,vendor_tag_name,vendor_category_en_Sweets & Bakes,primary_tags_number
46,196,-1.787413,0.006934,0.7,10,0,17,0,1,11,4.4,NaN,True,NaN
55,231,205.242327,44.210807,0.7,10,1,15,0,1,11,4.3,NaN,True,NaN
57,243,2.215692,0.714018,0.7,15,0,11,0,1,11,4.4,NaN,False,NaN


- Only 3 rows out of 100 have both vendor_tag_name and primary_tags_number missing.
Filling them with random values might create wrong tags.
Since they are very few and both are missing (not even tag number is present), dropping them is a safer option.

In [136]:
vendors.dropna(subset=['vendor_tag_name'], inplace=True)

In [137]:
#vendors[vendors[['vendor_tag_name','primary_tags_number']]['primary_tags_number'].isnull()][['vendor_tag_name','primary_tags_number']]

- For primary_tags_number, let’s think carefully:
- It tells how many tags/cuisines a restaurant has.
- But you already have vendor_category_en which lists all the cuisines.
- In a recommendation model, the actual cuisines matter, not just the count.

In [138]:
vendors.drop('primary_tags_number', axis=1, inplace=True)

In [139]:
vendors.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97 entries, 0 to 99
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 97 non-null     int64  
 1   latitude                           97 non-null     float64
 2   longitude                          97 non-null     float64
 3   delivery_charge                    97 non-null     float64
 4   serving_distance                   97 non-null     int64  
 5   is_open                            97 non-null     int64  
 6   prepration_time                    97 non-null     int64  
 7   discount_percentage                97 non-null     int64  
 8   verified                           97 non-null     int64  
 9   rank                               97 non-null     int64  
 10  vendor_rating                      97 non-null     float64
 11  vendor_tag_name                    97 non-null     object 
 12  v

# combine tables 

In [140]:
# 1 Start with train_orders as main table

merged_df = train_orders.copy()

In [141]:
#  Merge customer info
merged_df = merged_df.merge(train_customers, on='customer_id', how='left')


In [142]:
#  Merge location info
merged_df = merged_df.merge(
    train_locations,
    left_on=['customer_id', 'LOCATION_NUMBER'],
    right_on=['customer_id', 'location_number'],
    how='left'
)

In [143]:
#  Merge vendor info
merged_df = merged_df.merge(
    vendors,
    left_on='vendor_id',
    right_on='id',
    how='left',
    suffixes=('', '_vendor')
)

In [144]:
# Optional: drop duplicate columns like 'id' or 'location_number' if you want
#merged_df = merged_df.drop(columns=['id', 'location_number'])


In [145]:
merged_df.head(2)

,customer_id,item_count,grand_total,vendor_discount_amount,is_favorite,is_rated,vendor_rating,deliverydistance,vendor_id,LOCATION_NUMBER,has_vendor_rating,location_number,latitude,longitude,id,latitude_vendor,longitude_vendor,delivery_charge,serving_distance,is_open,prepration_time,discount_percentage,verified,rank,vendor_rating_vendor,vendor_tag_name,vendor_category_en_Sweets & Bakes
0,KL09J9N,6.0,10.1,0.0,0,0,0.0,0.0,84,0,0,0.0,-0.09065,-78.580196,84.0,-1.004923,0.078736,0.0,15.0,1.0,14.0,0.0,1.0,11.0,4.3,"Burgers,Fries,Kids meal,Shawarma",False
1,H5LGGFX,3.0,8.4,0.0,0,0,0.0,0.0,78,0,0,0.0,1.73395,-78.795830,78.0,-0.555404,0.196336,0.7,15.0,0.0,17.0,0.0,1.0,11.0,4.4,"Pizzas,Italian,Breakfast,Soups,Pasta,Salads,De...",False


In [146]:
merged_df.shape

(135221, 27)

In [147]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135221 entries, 0 to 135220
Data columns (total 27 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   customer_id                        135221 non-null  object 
 1   item_count                         135221 non-null  float64
 2   grand_total                        135221 non-null  float64
 3   vendor_discount_amount             135221 non-null  float64
 4   is_favorite                        135221 non-null  int64  
 5   is_rated                           135221 non-null  int64  
 6   vendor_rating                      135221 non-null  float64
 7   deliverydistance                   135221 non-null  float64
 8   vendor_id                          135221 non-null  int64  
 9   LOCATION_NUMBER                    135221 non-null  int64  
 10  has_vendor_rating                  135221 non-null  int32  
 11  location_number                    1352

In [148]:
merged_df[['customer_id','id']]

,customer_id,id
0,KL09J9N,84.0
1,H5LGGFX,78.0
2,CYLZB6T,4.0
3,4YKUKYN,157.0
4,WDNU30K,160.0
...,...,...
135216,YZNHLAP,106.0
135217,OD336VP,105.0
135218,N4F5VWP,66.0
135219,QURUHAI,81.0


# Test 

In [179]:
test_customers=pd.read_csv("Test_soulpage/test_customers.csv")
test_locations=pd.read_csv("Test_soulpage/test_locations.csv")

In [180]:
test_customers.head()

,customer_id,gender,dob,status,verified,language,created_at,updated_at
0,ICE2DJP,Male,NaN,1,1,EN,2/7/2023 16:45,2/7/2023 16:45
1,FWNUI71,Male,NaN,1,1,EN,3/22/2023 20:11,3/22/2023 20:11
2,LRX7BCH,Male,NaN,1,1,EN,4/17/2023 20:01,4/17/2023 20:01
3,D96DHMD,Male,NaN,1,1,EN,4/29/2023 22:35,4/29/2023 22:35
4,88Q8Y5V,Male,1997.0,1,1,EN,5/5/2023 12:38,5/5/2023 12:38


In [181]:
test_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9768 entries, 0 to 9767
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   customer_id  9768 non-null   object 
 1   gender       6321 non-null   object 
 2   dob          848 non-null    float64
 3   status       9768 non-null   int64  
 4   verified     9768 non-null   int64  
 5   language     5928 non-null   object 
 6   created_at   9768 non-null   object 
 7   updated_at   9768 non-null   object 
dtypes: float64(1), int64(2), object(5)
memory usage: 610.6+ KB


In [182]:
# Dropping unnecessary columns
test_customers.drop(['gender','dob','status','verified','language','created_at','updated_at'], axis=1, inplace=True)

In [183]:
# Check duplicates
test_customers['customer_id'].duplicated().sum()

15

In [184]:
test_customers.drop_duplicates(subset='customer_id', inplace=True)

In [185]:
test_customers.head(2)

,customer_id
0,ICE2DJP
1,FWNUI71


In [186]:
test_locations.head(2)

,customer_id,location_number,location_type,latitude,longitude
0,Z59FTQD,0,NaN,126.032278,-9.106019
1,0JP29SK,0,Home,0.278709,-78.623847


In [187]:
test_locations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16720 entries, 0 to 16719
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      16720 non-null  object 
 1   location_number  16720 non-null  int64  
 2   location_type    9070 non-null   object 
 3   latitude         16717 non-null  float64
 4   longitude        16717 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 653.3+ KB


In [188]:
# Let’s now check if any duplicate customer_id + location_number pairs exist in your train_locations table.
dup_count = test_locations.duplicated(subset=['customer_id','location_number']).sum()
print("Number of duplicate pairs:", dup_count)

Number of duplicate pairs: 0


In [189]:
# Drop rows where either latitude OR longitude is null
test_locations.dropna(subset=['latitude', 'longitude'], inplace=True)

In [190]:
# Reset index after dropping
test_locations.reset_index(drop=True, inplace=True)

In [191]:
test_locations.drop(['location_type'], axis=1, inplace=True)

In [192]:
test_locations.head(2)

,customer_id,location_number,latitude,longitude
0,Z59FTQD,0,126.032278,-9.106019
1,0JP29SK,0,0.278709,-78.623847


In [193]:
# Merge test_customers with test_locations on 'customer_id'
test_data = pd.merge(test_customers, test_locations, on='customer_id', how='left')
test_data.head(2)

,customer_id,location_number,latitude,longitude
0,ICE2DJP,0.0,-96.407538,-67.197291
1,ICE2DJP,1.0,0.038654,-78.595477


In [194]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16313 entries, 0 to 16312
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      16313 non-null  object 
 1   location_number  16312 non-null  float64
 2   latitude         16312 non-null  float64
 3   longitude        16312 non-null  float64
dtypes: float64(3), object(1)
memory usage: 509.9+ KB


In [195]:
test_data[test_data['location_number'].isnull()]

,customer_id,location_number,latitude,longitude
2442,7.37E+10,NaN,NaN,NaN


In [196]:
test_data.drop([2442], inplace=True)

In [197]:
test_data.reset_index(drop=True, inplace=True)

In [198]:
test_data.head(10)

,customer_id,location_number,latitude,longitude
0,ICE2DJP,0.0,-96.407538,-67.197291
1,ICE2DJP,1.0,0.038654,-78.595477
2,ICE2DJP,2.0,-95.106078,43.684151
3,ICE2DJP,3.0,-96.407677,43.557974
4,ICE2DJP,4.0,204.621247,44.091518
5,ICE2DJP,5.0,205.006635,44.048238
6,FWNUI71,0.0,-0.123325,-78.576334
7,FWNUI71,1.0,-0.485723,-78.533506
8,FWNUI71,2.0,-0.122362,0.642634
9,LRX7BCH,0.0,0.112173,-78.604166


In [199]:
test_locations.duplicated(subset=['customer_id','latitude','longitude']).sum()


0

In [200]:
# Remove duplicate locations (same customer_id + same coordinates)
#test_locations_unique = test_locations.drop_duplicates(subset=['customer_id', 'latitude', 'longitude'])

# Modelling 

- To build a simple recommendation system that suggests the nearest vendors (restaurants) to customers based on their geographical coordinates (latitude and longitude).

In [201]:
# Drop rows where vendor coordinates are missing
train_df = merged_df.dropna(subset=['latitude_vendor','longitude_vendor','vendor_id']).copy()

- We drop rows that have missing coordinates because distance cannot be calculated without them. 

In [202]:
# Drop rows where customer coordinates are missing (better than filling 0)
train_df = train_df.dropna(subset=['latitude','longitude'])

- Radians are required to calculate distances using the Haversine formula. 

In [203]:
# Convert coordinates to radians for Haversine
train_df['lat_cust_rad'] = np.radians(train_df['latitude'])
train_df['lon_cust_rad'] = np.radians(train_df['longitude'])
train_df['lat_ven_rad'] = np.radians(train_df['latitude_vendor'])
train_df['lon_ven_rad'] = np.radians(train_df['longitude_vendor'])

In [204]:
# Target column for historical orders
train_df['target'] = 0

- This function calculates the distance between two points on the Earth’s surface. 

In [205]:
def haversine_vec(lat1, lon1, lat2, lon2):
    """
    Vectorized haversine distance in km.
    lat1, lon1: arrays of test points
    lat2, lon2: arrays of vendor points
    """
    R = 6371  # Earth radius in km
    dlat = lat2 - lat1[:, None]   # broadcast lat1 over lat2
    dlon = lon2 - lon1[:, None]
    a = np.sin(dlat/2)**2 + np.cos(lat1[:, None])*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c


**Predict Closest Vendor for Each Test Customer**

In [206]:
test_lat_rad = np.radians(test_data['latitude'].values)
test_lon_rad = np.radians(test_data['longitude'].values)

vendor_lat_rad = train_df['lat_ven_rad'].values
vendor_lon_rad = train_df['lon_ven_rad'].values

In [210]:
submission = []

# Convert vendor coordinates to radians once
vendor_lat_rad = np.radians(train_df['latitude_vendor'].values)
vendor_lon_rad = np.radians(train_df['longitude_vendor'].values)

R = 6371  # Earth radius in km

def haversine_single(lat1, lon1, lat2_arr, lon2_arr):
    """
    Compute distance from single point to all vendors.
    """
    dlat = lat2_arr - lat1
    dlon = lon2_arr - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2_arr) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Iterate test customers one by one
for idx, row in test_data.iterrows():
    lat_cust = np.radians(row['latitude'])
    lon_cust = np.radians(row['longitude'])
    
    distances = haversine_single(lat_cust, lon_cust, vendor_lat_rad, vendor_lon_rad)
    
    # Find closest vendor
    top_idx = np.argmin(distances)
    
    submission.append({
        'CID': row['customer_id'],
        'LOC_NUM': int(train_df.iloc[top_idx]['location_number']),
        'VENDOR': int(train_df.iloc[top_idx]['vendor_id']),
        'target': 0
    })

submission_df = pd.DataFrame(submission)
submission_df.to_csv('submission.csv', index=False)



**Generate Top-N Vendor Recommendations**

In [211]:
import csv
import numpy as np

TOP_N = 5
max_loc = int(test_data['location_number'].max())

# Unique vendors only
vendor_df = train_df[['vendor_id','latitude_vendor','longitude_vendor','location_number']].drop_duplicates('vendor_id')

vendor_lat_rad = np.radians(vendor_df['latitude_vendor'].values)
vendor_lon_rad = np.radians(vendor_df['longitude_vendor'].values)
R = 6371

def haversine_single(lat1, lon1, lat2_arr, lon2_arr):
    dlat = lat2_arr - lat1
    dlon = lon2_arr - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2_arr) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Write CSV
with open('submission_topN.csv', mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['CID','LOC_NUM','VENDOR','target'])
    writer.writeheader()

    for idx, row in test_data.iterrows():
        cust_id = row['customer_id']
        lat_cust = np.radians(row['latitude'])
        lon_cust = np.radians(row['longitude'])

        # Distance to all unique vendors
        distances = haversine_single(lat_cust, lon_cust, vendor_lat_rad, vendor_lon_rad)
        top_indices = np.argsort(distances)[:TOP_N]

        for loc_num in range(max_loc + 1):
            for idx_vendor in top_indices:
                writer.writerow({
                    'CID': cust_id,
                    'LOC_NUM': loc_num,
                    'VENDOR': int(vendor_df.iloc[idx_vendor]['vendor_id']),
                    'target': 0
                })


**Testing for One Customer**

In [212]:
# Test for first customer
row = test_data.iloc[10]  # first customer
cust_id = row['customer_id']
lat_cust = np.radians(row['latitude'])
lon_cust = np.radians(row['longitude'])

distances = haversine_single(lat_cust, lon_cust, vendor_lat_rad, vendor_lon_rad)
top_indices = np.argsort(distances)[:TOP_N]

print(f"Customer ID: {cust_id}")
for loc_num in range(max_loc + 1):
    print(f"Location {loc_num}:")
    for idx_vendor in top_indices:
        vendor_id = int(train_df.iloc[idx_vendor]['vendor_id'])
        dist_km = distances[idx_vendor]
        print(f"  Vendor: {vendor_id}, Distance: {dist_km:.2f} km")

Customer ID: LRX7BCH
Location 0:
  Vendor: 82, Distance: 7.68 km
  Vendor: 157, Distance: 7.78 km
  Vendor: 310, Distance: 9.86 km
  Vendor: 157, Distance: 9.93 km
  Vendor: 265, Distance: 9.93 km
Location 1:
  Vendor: 82, Distance: 7.68 km
  Vendor: 157, Distance: 7.78 km
  Vendor: 310, Distance: 9.86 km
  Vendor: 157, Distance: 9.93 km
  Vendor: 265, Distance: 9.93 km
Location 2:
  Vendor: 82, Distance: 7.68 km
  Vendor: 157, Distance: 7.78 km
  Vendor: 310, Distance: 9.86 km
  Vendor: 157, Distance: 9.93 km
  Vendor: 265, Distance: 9.93 km
Location 3:
  Vendor: 82, Distance: 7.68 km
  Vendor: 157, Distance: 7.78 km
  Vendor: 310, Distance: 9.86 km
  Vendor: 157, Distance: 9.93 km
  Vendor: 265, Distance: 9.93 km
Location 4:
  Vendor: 82, Distance: 7.68 km
  Vendor: 157, Distance: 7.78 km
  Vendor: 310, Distance: 9.86 km
  Vendor: 157, Distance: 9.93 km
  Vendor: 265, Distance: 9.93 km
Location 5:
  Vendor: 82, Distance: 7.68 km
  Vendor: 157, Distance: 7.78 km
  Vendor: 310, Distanc

- In our case, all those locations have the same or very close coordinates (latitude & longitude).
- So the nearest 5 vendors for each location become the same vendors.
- That’s why you are seeing same 5 vendors repeated for 12 locations of the same customer.